In [ ]:
import pandas as pd
import os
import numpy as np
import random
import cv2
import sys
import shutil
from PIL import Image

In [ ]:
import tensorflow as tf
import math
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import albumentations as A
from albumentations.core.composition import Compose, OneOf
from tensorflow import keras

from sklearn.model_selection import StratifiedKFold, train_test_split
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB3, InceptionV3
from tensorflow.keras.utils import array_to_img, load_img, img_to_array

## Configuration

In [ ]:
train_data_dir = './train'
tardet_input_size = (256,256)
init_lr = 1e-4
reduce_lr_in = 3
epoch = 40
batch_size = 32
setps_in_epoch = 822
validation_steps = 245
best_ck_point = 'best_checkpoint.hdf5'
final_model = 'model.hdf5'

## Basic croping of all training set provided to increase sample sizes

cropping strategy is done in this notebook: https://www.kaggle.com/code/asheniranga/128-128-sorghum-cultivar-pre-process

## Pre-processing Pipeline

In [ ]:
def resize(image, size):
    return tf.image.resize(image, size)


def blur(img, blur_limit):
    return cv2.blur(img, ksize=[blur_limit, blur_limit])


def gaussian_blur(img, blur_limit=(3, 7), sigma_limit=0):
    return cv2.GaussianBlur(img, ksize=blur_limit, sigmaX=sigma_limit)


def motion_blur(img, blur_limit=7):
    kmb = np.zeros((blur_limit, blur_limit))
    kmb[(blur_limit - 1) // 2, :] = np.ones(blur_limit)
    kmb = kmb / blur_limit
    return cv2.filter2D(img, -1, kernel=kmb)


def gaussian_noise(img):
    x = tf.compat.v1.placeholder(dtype=tf.float32, shape=[512, 512, 3])
    noise = tf.random.normal(shape=tf.shape(x), mean=0.0, stddev=1, dtype=tf.float32)
    return tf.add(img, noise)


def iso_noise(img, color_shift=0.05, intensity=0.5):
    one_over_255 = float(1.0 / 255.0)
    image = np.multiply(img, one_over_255, dtype=np.float32)
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    _, stddev = cv2.meanStdDev(hls)

    luminance_noise = np.random.poisson(stddev[1] * intensity * 255, hls.shape[:2])
    color_noise = np.random.normal(0, color_shift * 360 * intensity, hls.shape[:2])

    hue = hls[..., 0]
    hue += color_noise
    hue[hue < 0] += 360
    hue[hue > 360] -= 360

    luminance = hls[..., 1]
    luminance += (luminance_noise / 255) * (1.0 - luminance)

    image = cv2.cvtColor(hls, cv2.COLOR_HLS2RGB) * 255
    return image.astype(np.uint8)


def random_cut_out(images):
    return tfa.image.random_cutout(images, (32, 32), constant_values=0)

In [ ]:
def aug_fn(image):
    data = {"image":image}
    aug_data = get_transform(**data)
    aug_img = aug_data["image"]
    aug_img = tf.cast(aug_img/255.0, tf.float32)
    aug_img = tf.image.resize(aug_img, size=[256, 256])
    return aug_img

get_transform = Compose([A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)])

def get_transforms_train(image):
    # get random crop of random crop window size
    crop_side = int(256*random.uniform(0.33, 1))
    temp = tf.image.random_crop(image, size=(crop_side, crop_side, 3)).numpy()
    temp = resize(temp, size=(256, 256)).numpy()

    # random flip (vertically)
    temp = tf.image.random_flip_left_right(temp).numpy()

    if np.random.choice([True, False], p=[0.6, 0.4]):
        # rotate randomly by N*90 deg
        k = random.randint(1, 4)
        temp = tf.image.rot90(temp, k=k).numpy()

    if np.random.choice([True, False], p=[0.45, 0.55]):
        if random.choice([True, False]):
            delta = random.uniform(-0.4, 0.4)
            cf = random.uniform(-1.5, 2.5)
            temp = tf.image.adjust_brightness(temp, delta=delta).numpy()
            temp = tf.image.adjust_contrast(temp, contrast_factor=cf).numpy()
        else:
            gamma = random.uniform(0.33, 2.2)
            temp = tf.image.adjust_gamma(temp, gamma=gamma).numpy()

    if np.random.choice([True, False], p=[0.25, 0.75]):
        delta = random.uniform(-0.2, 0.4)
        temp = tf.image.adjust_hue(temp, delta=delta).numpy()

    if np.random.choice([True, False], p=[0.2, 0.8]):
        sf = random.uniform(-0.2, 0.8)
        temp = tf.image.adjust_saturation(temp, saturation_factor=sf).numpy()

    if np.random.choice([True, False], p=[0.4, 0.6]):
        one_of_blur = random.choice([1, 2, 3])

        if one_of_blur == 1:
            temp = blur(temp, blur_limit=7)
        elif one_of_blur == 2:
            temp = gaussian_blur(temp)
        elif one_of_blur == 3:
            temp = motion_blur(temp)

    if np.random.choice([True, False], p=[0.35, 0.65]):
        temp = iso_noise(temp)

    if np.random.choice([True, False], p=[0.3, 0.7]):
        temp = temp.reshape([1,temp.shape[0], temp.shape[1], 3])
        temp = random_cut_out(temp).numpy()

        return tf.convert_to_tensor(temp[0]/255.0, dtype=tf.float32)

    temp = aug_fn(temp).numpy()

    return tf.convert_to_tensor(temp/255.0, dtype=tf.float32)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=[18, 6], dpi=300)
axes = axes.ravel()

for i in range(5):
    axes[i].imshow(array_to_img(get_transforms_train(img_to_array(load_img('../input/sorghum-cultivar-identification-512512/train/2017-06-01__10-26-27-479.png', target_size=(256,256))))))

plt.show()

### Split data

In [ ]:
df_train = pd.read_csv('../input/128128-sorghum-cultivar/train_meta.csv')
df_valid = pd.read_csv('../input/128128-sorghum-cultivar/valid_meta.csv')
    
df_valid, df_test = train_test_split(df_valid, test_size=0.1)

print(f"train size: {len(df_train)}")
print(f"valid size: {len(df_valid)}")
print(f"test size: {len(df_test)}")

print(df_train.cultivar.value_counts())
print(df_valid.cultivar.value_counts())
print(df_test.cultivar.value_counts())

In [ ]:
if not os.path.isdir('train'):
    os.mkdir('train')
    
for i, file in enumerate(os.listdir('../input/128128-sorghum-cultivar/train')):
    src = os.path.join('../input/128128-sorghum-cultivar/train', file)
    dst = os.path.join('./train', file)
    
    shutil.copyfile(src, dst)
    
    print(f"{i}/{len(os.listdir('../input/128128-sorghum-cultivar/train'))}", end='\r')

In [ ]:
c = 0
new_train_meta = []

for filename, label in df_train.sample(n=int(df_train.shape[0]*0.65)).values:
    if filename in os.listdir('train'):
        if random.choice([True, False]):
            image = tf.keras.utils.img_to_array(Image.open(os.path.join('train', filename)))
            
            c += 1
            print(f'{c}~{int(df_train.shape[0]*0.65)*0.5}', end='\r')
            
            # apply aumentations
            for i in range(5):
                process = get_transforms_train(image=image)
                
                dst_file = f'{i}-{filename}'
                tf.keras.utils.array_to_img(process).save(f'train/{dst_file}')
                new_train_meta.append([dst_file, label])


train_df_1 = pd.DataFrame(new_train_meta, columns=['image', 'cultivar'])
train_df_1

In [ ]:
df_train = pd.concat([df_train, train_df_1], ignore_index=True)
df_train

In [ ]:
temp = df_train['image'].sample(n=36).tolist()
fig, axes = plt.subplots(nrows=6, ncols=6, figsize=[18, 18], dpi=300)
axes = axes.ravel()

for i in range(36):
    axes[i].imshow(Image.open(f'train/{temp[i]}'))

plt.show()

## Training


In [ ]:
model_base = EfficientNetB3(include_top=False, input_shape=(256, 256, 3), weights='imagenet')

In [ ]:
input_layer = Input(shape=(256, 256, 3))
x_ = model_base(input_layer)
x_ = GlobalAveragePooling2D()(x_)
output_layer = Dense(units=100, activation='softmax')(x_)

model = Model(input_layer, output_layer)

In [ ]:
model.compile(optimizer=Adam(learning_rate=init_lr),
              loss=categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
train_generator = ImageDataGenerator(samplewise_center=False,
                                     samplewise_std_normalization=False,
                                     width_shift_range=0.35,
                                     height_shift_range=0.45,
                                     channel_shift_range=0.65,
                                     fill_mode='reflect').flow_from_dataframe(dataframe=df_train,
                                                           directory=train_data_dir,
                                                           x_col='image',
                                                           y_col='cultivar',
                                                           batch_size=batch_size,
                                                           target_size=tardet_input_size)

valid_generator = ImageDataGenerator().flow_from_dataframe(dataframe=df_valid,
                                                           directory=train_data_dir,
                                                           x_col='image',
                                                           y_col='cultivar',
                                                           batch_size=batch_size,
                                                           target_size=tardet_input_size)

test_generator = ImageDataGenerator().flow_from_dataframe(dataframe=df_test,
                                                          directory=train_data_dir,
                                                          x_col='image',
                                                          y_col='cultivar',
                                                          batch_size=batch_size,
                                                          target_size=tardet_input_size)

In [ ]:
K = keras.backend


class OneCycleLr(keras.callbacks.Callback):
    def __init__(self,
                 max_lr: float,
                 total_steps: int = None,
                 epochs: int = None,
                 steps_per_epoch: int = None,
                 pct_start: float = 0.2,
                 anneal_strategy: str = "cos",
                 cycle_momentum: bool = True,
                 base_momentum: float = 0.85,
                 max_momentum: float = 0.95,
                 div_factor: float = 1.0e+3,
                 final_div_factor: float = 1e4,
                 ) -> None:

        super(OneCycleLr, self).__init__()

        # validate total steps:
        if total_steps is None and epochs is None and steps_per_epoch is None:
            raise ValueError(
                "You must define either total_steps OR (epochs AND steps_per_epoch)"
            )
        elif total_steps is not None:
            if total_steps <= 0 or not isinstance(total_steps, int):
                raise ValueError(
                    "Expected non-negative integer total_steps, but got {}".format(
                        total_steps
                    )
                )
            self.total_steps = total_steps
        else:
            if epochs <= 0 or not isinstance(epochs, int):
                raise ValueError(
                    "Expected non-negative integer epochs, but got {}".format(
                        epochs)
                )
            if steps_per_epoch <= 0 or not isinstance(steps_per_epoch, int):
                raise ValueError(
                    "Expected non-negative integer steps_per_epoch, but got {}".format(
                        steps_per_epoch
                    )
                )
            # Compute total steps
            self.total_steps = epochs * steps_per_epoch

        self.step_num = 0
        self.step_size_up = float(pct_start * self.total_steps) - 1
        self.step_size_down = float(self.total_steps - self.step_size_up) - 1

        # Validate pct_start
        if pct_start < 0 or pct_start > 1 or not isinstance(pct_start, float):
            raise ValueError(
                "Expected float between 0 and 1 pct_start, but got {}".format(
                    pct_start)
            )

        # Validate anneal_strategy
        if anneal_strategy not in ["cos", "linear"]:
            raise ValueError(
                "anneal_strategy must by one of 'cos' or 'linear', instead got {}".format(
                    anneal_strategy
                )
            )
        elif anneal_strategy == "cos":
            self.anneal_func = self._annealing_cos
        elif anneal_strategy == "linear":
            self.anneal_func = self._annealing_linear

        # Initialize learning rate variables
        self.initial_lr = max_lr / div_factor
        self.max_lr = max_lr
        self.min_lr = self.initial_lr / final_div_factor

        # Initial momentum variables
        self.cycle_momentum = cycle_momentum
        if self.cycle_momentum:
            self.m_momentum = max_momentum
            self.momentum = max_momentum
            self.b_momentum = base_momentum

        # Initialize variable to learning_rate & momentum
        self.track_lr = []
        self.track_mom = []

    def _annealing_cos(self, start, end, pct) -> float:
        "Cosine anneal from `start` to `end` as pct goes from 0.0 to 1.0."
        cos_out = math.cos(math.pi * pct) + 1
        return end + (start - end) / 2.0 * cos_out

    def _annealing_linear(self, start, end, pct) -> float:
        "Linearly anneal from `start` to `end` as pct goes from 0.0 to 1.0."
        return (end - start) * pct + start

    def set_lr_mom(self) -> None:
        """Update the learning rate and momentum"""
        if self.step_num <= self.step_size_up:
            # update learining rate
            computed_lr = self.anneal_func(
                self.initial_lr, self.max_lr, self.step_num / self.step_size_up
            )
            K.set_value(self.model.optimizer.lr, computed_lr)
            # update momentum if cycle_momentum
            if self.cycle_momentum:
                computed_momentum = self.anneal_func(
                    self.m_momentum, self.b_momentum, self.step_num / self.step_size_up
                )
                try:
                    K.set_value(self.model.optimizer.momentum,
                                computed_momentum)
                except:
                    K.set_value(self.model.optimizer.beta_1, computed_momentum)
        else:
            down_step_num = self.step_num - self.step_size_up
            # update learning rate
            computed_lr = self.anneal_func(
                self.max_lr, self.min_lr, down_step_num / self.step_size_down
            )
            K.set_value(self.model.optimizer.lr, computed_lr)
            # update momentum if cycle_momentum
            if self.cycle_momentum:
                computed_momentum = self.anneal_func(
                    self.b_momentum,
                    self.m_momentum,
                    down_step_num / self.step_size_down,
                )
                try:
                    K.set_value(self.model.optimizer.momentum,
                                computed_momentum)
                except:
                    K.set_value(self.model.optimizer.beta_1, computed_momentum)

    def on_train_begin(self, logs=None) -> None:
        # Set initial learning rate & momentum values
        K.set_value(self.model.optimizer.lr, self.initial_lr)
        if self.cycle_momentum:
            try:
                K.set_value(self.model.optimizer.momentum, self.momentum)
            except:
                K.set_value(self.model.optimizer.beta_1, self.momentum)

    def on_train_batch_end(self, batch, logs=None) -> None:
        # Grab the current learning rate & momentum
        lr = float(K.get_value(self.model.optimizer.lr))
        try:
            mom = float(K.get_value(self.model.optimizer.momentum))
        except:
            mom = float(K.get_value(self.model.optimizer.beta_1))
        # Append to the list
        self.track_lr.append(lr)
        self.track_mom.append(mom)
        # Update learning rate & momentum
        self.set_lr_mom()
        # increment step_num
        self.step_num += 1

    def plot_lrs_moms(self, axes=None) -> None:
        if axes == None:
            _, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        else:
            try:
                ax1, ax2 = axes
            except:
                ax1, ax2 = axes[0], axes[1]
        ax1.plot(self.track_lr)
        ax1.set_title("Learning Rate vs Steps")
        ax2.plot(self.track_mom)
        ax2.set_title("Momentum (or beta_1) vs Steps")


model_checkpoint_callback = ModelCheckpoint(filepath=best_ck_point,
                                            save_weights_only=False,
                                            monitor='val_accuracy',
                                            mode='max',
                                            verbose=1,
                                            save_best_only=True)

early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           verbose=1,
                           restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              patience=reduce_lr_in,
                              verbose=1)

one_cycle = OneCycleLr(max_lr=1e-3, steps_per_epoch=setps_in_epoch, epochs=epoch)

In [ ]:
model.optimizer.lr

In [ ]:
history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=epoch,
                    steps_per_epoch=821,
                    validation_steps=370,
                    callbacks=[early_stop, model_checkpoint_callback, one_cycle])

In [ ]:
model.evaluate(valid_generator)

In [ ]:
temp_1 = pd.DataFrame(history.history)
temp_1.to_pickle('history.pkl')
temp_1

In [ ]:
model.save(final_model)

In [ ]:
load_model(final_model).evaluate(valid_generator)

In [ ]:
valid_generator.class_indices

In [ ]:
import json

with open('class_indices.json', 'w') as file:
    json.dump(train_generator.class_indices, file)

In [ ]:
df_pred = pd.read_csv('../input/sorghum-id-fgvc-9/sample_submission.csv')

In [ ]:
preds_generator = ImageDataGenerator().flow_from_dataframe(dataframe=df_pred,
                                                           directory='../input/sorghum-id-fgvc-9/test',
                                                           x_col='filename',
                                                           y_col='cultivar',
                                                           batch_size=batch_size,
                                                           target_size=tardet_input_size)

preds = model.predict(preds_generator)

In [ ]:
key = []

for i in range(len(preds)):
    key.append(list(valid_generator.class_indices.keys())[list(valid_generator.class_indices.values()).index(np.argmax(preds[i]))])


In [ ]:
df_pred = df_pred.drop('cultivar', axis=1)
df_pred['cultivar'] = key
df_pred

In [ ]:
df_pred.to_csv('submission_15.csv', index=False)